# Xây dựng cây quyết định và rừng cây trên dữ liệu bệnh tiểu đường.

## 1. Khai báo thư viện

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 2. Tải và khám phá dữ liệu

In [3]:
# Tải dữ liệu
df = pd.read_csv("data/diabetes_prediction_dataset.csv")

# Xem 5 dòng đầu tiên
print("Dữ liệu gốc:")
print(df.head())

# Xem thông tin các cột (kiểu dữ liệu, có bị thiếu không)
print("\nThông tin cột:")
df.info()

Dữ liệu gốc:
   gender   age  hypertension  heart_disease smoking_history    bmi  \
0  Female  80.0             0              1           never  25.19   
1  Female  54.0             0              0         No Info  27.32   
2    Male  28.0             0              0           never  27.32   
3  Female  36.0             0              0         current  23.45   
4    Male  76.0             1              1         current  20.14   

   HbA1c_level  blood_glucose_level  diabetes  
0          6.6                  140         0  
1          6.6                   80         0  
2          5.7                  158         0  
3          5.0                  155         0  
4          4.8                  155         0  

Thông tin cột:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  ob

## 3. Tiền xử lý dữ liệu

### a. Chuyển đổi dữ liệu Categorical (Chữ -> Số)

In [4]:
# Tự động tìm các cột có kiểu 'object' (chữ) và chuyển đổi chúng
# drop_first=True để tránh bẫy biến giả (dummy variable trap)
df_processed = pd.get_dummies(df, drop_first=True)

# Xem kết quả sau khi chuyển đổi
print("\nDữ liệu sau khi One-Hot Encoding:")
print(df_processed.head())


Dữ liệu sau khi One-Hot Encoding:
    age  hypertension  heart_disease    bmi  HbA1c_level  blood_glucose_level  \
0  80.0             0              1  25.19          6.6                  140   
1  54.0             0              0  27.32          6.6                   80   
2  28.0             0              0  27.32          5.7                  158   
3  36.0             0              0  23.45          5.0                  155   
4  76.0             1              1  20.14          4.8                  155   

   diabetes  gender_Male  gender_Other  smoking_history_current  \
0         0        False         False                    False   
1         0        False         False                    False   
2         0         True         False                    False   
3         0        False         False                     True   
4         0         True         False                     True   

   smoking_history_ever  smoking_history_former  smoking_history_never  \
0

### b. Tách biến X (đặc trưng) và y (mục tiêu)

In [5]:
# y là cột mục tiêu
y = df_processed['diabetes']

# X là tất cả các cột còn lại (bỏ cột 'diabetes')
X = df_processed.drop('diabetes', axis=1)

### c. Chia dữ liệu Train / Test

In [6]:
# Chia 80% cho huấn luyện, 20% cho kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### d. Chuẩn hóa dữ liệu (Scaling)

In [7]:
# Tạo đối tượng scaler
scaler = StandardScaler()

# Fit và transform trên X_train
X_train_scaled = scaler.fit_transform(X_train)

# Chỉ transform trên X_test (dùng mean/std từ X_train)
X_test_scaled = scaler.transform(X_test)

## 4. Huấn luyện mô hình

In [8]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
best_svm = None
best_val_acc = -1
best_kernel = None

print("\nBắt đầu huấn luyện SVM với các kernel...")

for i in range(4):
    # Khởi tạo mô hình SVM với kernel tương ứng
    clf = svm.SVC(kernel=kernels[i], probability=True)
    clf.fit(X_train_scaled, y_train)
    tmp_val_acc = clf.score(X_test_scaled, y_test)
    
    print(f"Kernel: {kernels[i]}, Accuracy: {tmp_val_acc}")
    
    # Cập nhật mô hình tốt nhất
    if (tmp_val_acc > best_val_acc):
        best_val_acc = tmp_val_acc
        best_svm = clf
        best_kernel = kernels[i]

# In ra kết quả tốt nhất
print("\n--- Kết quả ---")
print(f"Best validation accuracy: {best_val_acc} với kernel: {best_kernel}")


Bắt đầu huấn luyện SVM với các kernel...
Kernel: linear, Accuracy: 0.95935
Kernel: poly, Accuracy: 0.9625
Kernel: rbf, Accuracy: 0.96135
Kernel: sigmoid, Accuracy: 0.91955

--- Kết quả ---
Best validation accuracy: 0.9625 với kernel: poly


## 5. Đánh giá chi tiết mô hình tốt nhất

In [9]:
# Dùng mô hình tốt nhất để dự đoán trên tập test
y_pred = best_svm.predict(X_test_scaled)

# In báo cáo phân loại (precision, recall, f1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# In ma trận nhầm lẫn
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     18292
           1       0.98      0.57      0.72      1708

    accuracy                           0.96     20000
   macro avg       0.97      0.79      0.85     20000
weighted avg       0.96      0.96      0.96     20000


Confusion Matrix:
[[18269    23]
 [  727   981]]
